<a href="https://colab.research.google.com/github/subhamsahoo-4/Amazon_product_review/blob/main/aids_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving APR dataset.zip to APR dataset.zip


In [2]:
import zipfile
import os

zip_path = "/content/APR dataset.zip"  # Adjust if your filename is different
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Optional: List extracted files
import os
print(os.listdir(extract_path))

['Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv', '1429_1.csv', 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv']


In [3]:
# Step 1: Load the datasets
import pandas as pd
import os

df1 = pd.read_csv("/content/dataset/1429_1.csv", low_memory=False)
df2 = pd.read_csv("/content/dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv", low_memory=False)
df3 = pd.read_csv("/content/dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv", low_memory=False)

# Step 2: Select relevant columns and combine
common_cols = ['reviews.text', 'reviews.rating']
df1 = df1[common_cols]
df2 = df2[common_cols]
df3 = df3[common_cols]

df = pd.concat([df1, df2, df3], ignore_index=True)
df.dropna(subset=['reviews.text', 'reviews.rating'], inplace=True)

# Step 3: Convert rating to binary sentiment
def label_sentiment(rating):
    try:
        rating = float(rating)
        if rating >= 4:
            return 1
        elif rating <= 2:
            return 0
        else:
            return None  # Neutral
    except:
        return None

df['sentiment'] = df['reviews.rating'].apply(label_sentiment)
df.dropna(subset=['sentiment'], inplace=True)

# Step 4: Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['reviews.text'], df['sentiment'], test_size=0.2, random_state=42
)

# Step 5: Vectorize text using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Step 6: Train the model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Step 7: Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test_tfidf)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))

✅ Accuracy: 0.9739471257300953

📊 Classification Report:
               precision    recall  f1-score   support

         0.0       0.90      0.35      0.51       494
         1.0       0.98      1.00      0.99     12518

    accuracy                           0.97     13012
   macro avg       0.94      0.68      0.75     13012
weighted avg       0.97      0.97      0.97     13012



In [4]:
sample_reviews = [
    "Absolutely love this product. Highly recommend!",
    "Terrible. It broke after one day.",
    "It's okay, nothing special.",
    "Below average product, not same as shown"
]

sample_tfidf = vectorizer.transform(sample_reviews)
predictions = model.predict(sample_tfidf)

for review, pred in zip(sample_reviews, predictions):
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

Review: Absolutely love this product. Highly recommend!
Predicted Sentiment: Positive

Review: Terrible. It broke after one day.
Predicted Sentiment: Negative

Review: It's okay, nothing special.
Predicted Sentiment: Positive

Review: Below average product, not same as shown
Predicted Sentiment: Positive

